In [1]:
import torch

In [ ]:
torch.__version__

'1.12.0.dev20220426+cu113'

In [3]:
torch.__file__

'/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/__init__.py'

In [2]:
from config.defaults import train_config

In [3]:
cfg = train_config()

In [4]:
torch.cuda.empty_cache()

In [6]:
model_name = "google/t5-v1_1-xl"
tokenizer_name = "t5-large"

In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead,AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [8]:
model.load_state_dict(torch.load("demo_xl_0061vl.pt")) #("t5-smalltrain_acc4.pt"))


<All keys matched successfully>

In [43]:
model.eval();

In [10]:
import datasets_grammar as dg
from config.defaults import train_config
import os

In [11]:
cfg = train_config()
torch.cuda.set_device("cuda:0")

In [12]:
test_dataset = dg.get_dataset(tokenizer, cfg.dataset_test, 512, 512, True)
print(len(test_dataset))
print(f"using dataset {cfg.dataset_test}")

Using custom data configuration default-d7ffb4b425f4541a
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-d7ffb4b425f4541a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

2988
using dataset datasets_grammar/grammar_validation.csv


In [13]:
def validate_one_step(model, batch):
    #print(batch.keys())
    curr_loss = torch.zeros(3)#.to("cuda:0")
    for key in batch.keys():
        #print(f"batch key = {key}")
        batch[key] = batch[key].to("cuda:0")
        
    output = model(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=batch["target_ids"],
        )
    curr_loss[0] += output["loss"].item()  # sum up batch loss
    curr_loss[1] +=len(batch)
        #print(curr_loss)
        #pred = output.logits.argmax(
         #       dim=1, keepdim=True
         #   )  # get the index of the max log-probability
        #print(pred)
        #curr_loss[1] += pred.eq(batch["target_ids"].view_as(pred)).sum().item()
        #curr_loss[2] += len(batch)
        #print(output)
        #loss = 0
    #loss = model(**data)
    return curr_loss


In [38]:
def validate_one_epoch(model, data_loader):
    model.eval()
    total_loss = torch.zeros(3)
    for batch_index, data in enumerate(data_loader):
        with torch.no_grad():
            loss = validate_one_step(model, data)
        total_loss += loss
        
        print(total_loss[0]/total_loss[1])
        if batch_index > 4:
            break
    return total_loss

In [61]:
def test_one_epoch(model, dataloader, device):
    ddp_loss = torch.zeros(3).to("cuda:0")
    with torch.no_grad():
        for batch in dataloader:
            for key in batch.keys():
                batch[key] = batch[key].to("cuda:0")
            output = model(
                input_ids=batch["source_ids"],
                attention_mask=batch["source_mask"],
                labels=batch["target_ids"],
            )
            ddp_loss[0] += output["loss"].item()  # sum up batch loss
            ddp_loss[1] += len(batch)
            #ddp_loss[0] += 
            #pred = output.logits.argmax(
             #   dim=1, keepdim=True
            #)  # get the index of the max log-probability
            #print(f"pred = {pred.shape}\n{pred}")
            #ddp_loss[1] += pred.eq(batch["target_ids"].view_as(pred)).sum().item()
            

    test_loss = ddp_loss[0] / ddp_loss[1]
    print(f"Total test loss: {test_loss}")
    return test_loss
    

In [54]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [55]:
device = torch.cuda.set_device("cuda:0")

In [62]:
test_loss = test_one_epoch(model,test_dataloader, device)

Total test loss: 3.035662889480591


In [63]:
test_loss

tensor(3.0357, device='cuda:0')

In [41]:
test_loss[0]/test_loss[1]

tensor(2.4083)

In [14]:
test = "The cherry blossoms was so pretty"
test2 = "Our dogs is running in the park."
test3 = "Their the ones who made a mistake."
test4 = "Its cold outside today."
test5 = "The baby was held by its mother."
test6 = "The book on AI really effected me."
test7= "The children love eating, coloring, and to play with their toys."

In [19]:
text = test #"I is reading about AI articles "
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')
#inputs.to("cuda:0")

output = model.generate(inputs['input_ids'], num_beams=5, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
result=("".join(correction))


In [21]:
def get_correction(model, tokenizer, text):
    inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')
    #inputs.to("cuda:0")

    output = model.generate(inputs['input_ids'], num_beams=5, max_length=512, early_stopping=True)
    correction=tokenizer.batch_decode(output, skip_special_tokens=True)
    result=("".join(correction))
    return result


In [107]:
def eval_string(input_string, model, tokenizer):
    print(fore.LIGHTBLUE_EX +f"{input_string}")
    result = get_correction(model, tokenizer, input_string)
    print(fore.GREEN+f"{result}\n")

In [20]:
print(f"input: {text}\n")
print(f"corrected: {result}")

input: The cherry blossoms was so pretty

corrected: The cherry blossoms were so pretty.


In [44]:
test11="Every girl must bring their own lunch."
test12= "Its a cold day for October."
test13 = "These recipes is good for beginning chefs."
test14 = "At eight years old, my father gave me a pony for Christmas."

In [54]:
test15 = "Sharon stayed home from school the other day. Because she was sick."
test16 = "Jim went to the store and Ella went with him."
test17 = "The dad found the boy, and he was happy.”"
test18 = "There father went to school there."
test19="The rain had a good affect on the farmer’s field."

In [64]:
test20="The dogs dish was full of bone’s."
test21="I am doing good in math."
test22="Susan gave me a real nice bouquet of flowers."
test23="There were less cans on the shelves than there were yesterday."
test24="When you get done with that lab report, can you send it to Bill and I?"

In [82]:
test25="We need to get our sale’s numbers up."
test26="He starts work everyday at 8 a.m."
test27 = "The marketing manager told Riley and I to talk with her."
test28 = "Less than 50 people showed up for the presentation."
test29="I could just lay down and go to sleep."
test30="The people that reach their sales target will get a reward."
test31="That presentation was better then the first one. "
test32="Your my favorite supervisor."
test33 = "There going to they’re office over their."

In [111]:
eval_string(test13, model, tokenizer)

These recipes is good for beginning chefs.
These recipes are good for beginning chefs.



In [ ]:
# test, test6, test7, test15 (sentence fragments), test19(affect effect), test13, 

In [86]:
import colorama

In [89]:
from colorama import Fore as fore

In [104]:
print(fore.BLUE+"hello")

hello


In [109]:
inputs = [test, test6, test7, test15, test19, test13]
outputs = []

for item in inputs:
    res = eval_string(item, model, tokenizer)
    outputs.append(res)

The cherry blossoms was so pretty
The cherry blossoms were so pretty.

The book on AI really effected me.
The book on AI really affected me.

The children love eating, coloring, and to play with their toys.
Children love eating, coloring, and playing with their toys.

Sharon stayed home from school the other day. Because she was sick.
Sharon stayed home from school the other day because she was sick.

The rain had a good affect on the farmer’s field.
The rain had a good effect on the farmer’s field.

These recipes is good for beginning chefs.
These recipes are good for beginning chefs.



In [101]:
for pre, post in zip(inputs, outputs):
    print(pre)
    print(post)
    print()

The cherry blossoms was so pretty
None

The book on AI really effected me.
None

The children love eating, coloring, and to play with their toys.
None

Sharon stayed home from school the other day. Because she was sick.
None

The rain had a good affect on the farmer’s field.
None

These recipes is good for beginning chefs.
None

